In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import re
import pandas as pd
import numpy as np
import time
import pickle
import hashlib
import mwparserfromhell
import json

In [2]:
controversial_title_dir = '/home/ubuntu/intermediate-result/controversial_titles/'

In [3]:
all_title_files = glob(controversial_title_dir + '*')

In [4]:
metric_path = '../../process-all-wiki/intermediate-result/'

In [5]:
all_files = glob(metric_path + '*')

In [ ]:
# get all controversial titles (talk pages)

In [6]:
talks = []
for i in all_title_files:
    with open(i, 'rb') as f:
        c_titles = pickle.load(f)
        talks += c_titles

In [7]:
# get corresponding article pages
articles = list(map(lambda x: str(x).replace('Talk:', ''), talks))

In [8]:
controversial_articles = articles + talks

In [9]:
# with open('result/' + 'controversial_articles.txt', "wb") as fp:   #Pickling
#     pickle.dump(controversial_articles, fp)

In [ ]:
# read all folders
# for controversial articles, get all metrics, and append it to a csv file

In [10]:
for file in all_files:
    test = ddf.read_parquet(os.path.join(metric_path,file,'revision.count')).compute()
    test = test.loc[test.index.isin(controversial_articles)]
    if test.empty:
        continue
    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'page.length')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'unique.editors')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'links.count')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'anon.edits')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'minor')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'reverts_new')).compute())

    test = test.join(ddf.read_parquet(os.path.join(metric_path,file,'crc')).compute())
    
    test.to_csv('controversial_articles.csv', mode='a', header=False)

In [ ]:
# convert the csv to pandas

In [11]:
data = pd.read_csv('controversial_articles.csv',names=['revision.count','page.length','unique.editors','links.count','anon.edits',
                                                       'minor','reverts','crc'])

In [ ]:
# merge article and talk pages
# rename metrics for talk pages (add prefix 'talk.')
# remove the leading 'Talk:' for talk pages
# then join talk and article pages to get all info for an article page

In [12]:
article = data[data['crc'].isnull()].drop(columns={'crc'})

In [13]:
talk = data[data['crc'].notnull()]

In [14]:
talk.index = talk.index.str.replace('Talk:','')

In [15]:
talk.columns = ['talk.' + str(col) for col in talk.columns]

In [16]:
final = article.join(talk).fillna(0).astype('int')

In [17]:
final.to_csv('final_data.csv',index=True)